# Import packages & functions

In [1]:
import os
import sys
import json
import argparse
import numpy as np
import math
from einops import rearrange
import time
import random
import string
import h5py
from tqdm import tqdm
import webdataset as wds
import gc
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import transforms
from accelerate import Accelerator
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
# SDXL unCLIP requires code from https://github.com/Stability-AI/generative-models/tree/main
sys.path.append('generative_models/')
import sgm
from models import Clipper
from versatile_diffusion import Reconstructor
from generative_models.sgm.modules.encoders.modules import FrozenOpenCLIPImageEmbedder # bigG embedder
# tf32 data type is faster than standard float32
torch.backends.cuda.matmul.allow_tf32 = True
from sklearn.linear_model import Ridge
import pickle
# custom functions #
import utils

In [ ]:
### Multi-GPU config ###
local_rank = os.getenv('RANK')
if local_rank is None: 
    local_rank = 0
else:
    local_rank = int(local_rank)
print("LOCAL RANK ", local_rank)  

data_type = torch.float16 # change depending on your mixed_precision
num_devices = torch.cuda.device_count()
if num_devices==0: num_devices = 1

# First use "accelerate config" in terminal and setup using deepspeed stage 2 with CPU offloading!
accelerator = Accelerator(split_batches=False, mixed_precision="fp16")
if utils.is_interactive(): # set batch size here if using interactive notebook instead of submitting job
    global_batch_size = batch_size = 8
else:
    global_batch_size = os.environ["GLOBAL_BATCH_SIZE"]
    batch_size = int(os.environ["GLOBAL_BATCH_SIZE"]) // num_devices

In [ ]:
print("PID of this process =",os.getpid())
device = accelerator.device
print("device:",device)
world_size = accelerator.state.num_processes
distributed = not accelerator.state.distributed_type == 'NO'
num_devices = torch.cuda.device_count()
if num_devices==0 or not distributed: num_devices = 1
num_workers = num_devices
print(accelerator.state)

print("distributed =",distributed, "num_devices =", num_devices, "local rank =", local_rank, "world size =", world_size, "data_type =", data_type)
print = accelerator.print # only print if local_rank=0

# Configurations

In [2]:
# if running this interactively, can specify jupyter_args here for argparser to use
if utils.is_interactive():
    model_name = "testing"
    print("model_name:", model_name)
    
    # global_batch_size and batch_size should already be defined in the 2nd cell block
    jupyter_args = f"--data_path=../dataset/ \
                    --cache_dir=../cache/ \
                    --model_name={model_name} \
                    --batch_size=64 \
                    --no-multi_subject --subj=1 --num_sessions=40 \
                    --hidden_dim=1024 --clip_scale=1. \
                    --no-blurry_recon --blur_scale=.5  \
                    --seq_past=0 --seq_future=0 \
                    --no-use_prior --prior_scale=30 \
                    --n_blocks=4 --max_lr=3e-4 --mixup_pct=.33 --num_epochs=150 --no-use_image_aug \
                    --ckpt_interval=1 --ckpt_saving"

    print(jupyter_args)
    jupyter_args = jupyter_args.split()
    
    from IPython.display import clear_output # function to clear print outputs in cell
    %load_ext autoreload 
    # this allows you to change functions in models.py or utils.py and have this notebook automatically update with your revisions
    %autoreload 2 

model_name: testing
--data_path=../dataset/                     --cache_dir=../cache/                     --model_name=testing                     --batch_size=64                     --no-multi_subject --subj=1 --num_sessions=40                     --hidden_dim=1024 --clip_scale=1.                     --no-blurry_recon --blur_scale=.5                      --seq_past=0 --seq_future=0                     --no-use_prior --prior_scale=30                     --n_blocks=4 --max_lr=3e-4 --mixup_pct=.33 --num_epochs=150 --no-use_image_aug                     --ckpt_interval=1 --ckpt_saving


In [3]:
parser = argparse.ArgumentParser(description="Model Training Configuration")
parser.add_argument(
    "--model_name", type=str, default="testing",
    help="name of model, used for ckpt saving and wandb logging (if enabled)",
)
parser.add_argument(
    "--data_path", type=str, default=os.getcwd(),
    help="Path to where NSD data is stored / where to download it to",
)
parser.add_argument(
    "--cache_dir", type=str, default=os.getcwd(),
    help="Path to where misc. files downloaded from huggingface are stored. Defaults to current src directory.",
)
parser.add_argument(
    "--subj",type=int, default=1, choices=[1,2,3,4,5,6,7,8,9,10,11],
    help="Validate on which subject?",
)
parser.add_argument(
    "--multisubject_ckpt", type=str, default=None,
    help="Path to pre-trained multisubject model to finetune a single subject from. multisubject must be False.",
)
parser.add_argument(
    "--num_sessions", type=int, default=40,
    help="Number of training sessions to include",
)
parser.add_argument(
    "--use_prior",action=argparse.BooleanOptionalAction,default=True,
    help="whether to train diffusion prior (True) or just rely on retrieval part of the pipeline (False)",
)
parser.add_argument(
    "--visualize_prior",action=argparse.BooleanOptionalAction,default=False,
    help="output visualizations from unCLIP every ckpt_interval (requires much more memory!)",
)
parser.add_argument(
    "--batch_size", type=int, default=16,
    help="Batch size can be increased by 10x if only training retreival submodule and not diffusion prior",
)
parser.add_argument(
    "--wandb_log",action=argparse.BooleanOptionalAction,default=False,
    help="whether to log to wandb",
)
parser.add_argument(
    "--resume_from_ckpt",action=argparse.BooleanOptionalAction,default=False,
    help="if not using wandb and want to resume from a ckpt",
)
parser.add_argument(
    "--wandb_project",type=str,default="mindeye_imagery",
    help="wandb project name",
)
parser.add_argument(
    "--mixup_pct",type=float,default=.33,
    help="proportion of way through training when to switch from BiMixCo to SoftCLIP",
)
parser.add_argument(
    "--blurry_recon",action=argparse.BooleanOptionalAction,default=True,
    help="whether to output blurry reconstructions",
)
parser.add_argument(
    "--blur_scale",type=float,default=.5,
    help="multiply loss from blurry recons by this number",
)
parser.add_argument(
    "--clip_scale",type=float,default=1.,
    help="multiply contrastive loss by this number",
)
parser.add_argument(
    "--prior_scale",type=float,default=30,
    help="multiply diffusion prior loss by this",
)
parser.add_argument(
    "--use_image_aug",action=argparse.BooleanOptionalAction,default=False,
    help="whether to use image augmentation",
)
parser.add_argument(
    "--num_epochs",type=int,default=150,
    help="number of epochs of training",
)
parser.add_argument(
    "--multi_subject",action=argparse.BooleanOptionalAction,default=False,
)
parser.add_argument(
    "--new_test",action=argparse.BooleanOptionalAction,default=True,
)
parser.add_argument(
    "--n_blocks",type=int,default=4,
)
parser.add_argument(
    "--hidden_dim",type=int,default=1024,
)
parser.add_argument(
    "--seq_past",type=int,default=0,
)
parser.add_argument(
    "--seq_future",type=int,default=0,
)
parser.add_argument(
    "--lr_scheduler_type",type=str,default='cycle',choices=['cycle','linear'],
)
parser.add_argument(
    "--ckpt_saving",action=argparse.BooleanOptionalAction,default=True,
)
parser.add_argument(
    "--ckpt_interval",type=int,default=5,
    help="save backup ckpt and reconstruct every x epochs",
)
parser.add_argument(
    "--seed",type=int,default=42,
)
parser.add_argument(
    "--max_lr",type=float,default=3e-4,
)
parser.add_argument(
    "--weight_decay",type=float,default=1e-2,
)
parser.add_argument(
    "--train_imageryrf",action=argparse.BooleanOptionalAction,default=False,
    help="Use the ImageryRF dataset for pretraining",
)
parser.add_argument(
    "--no_nsd",action=argparse.BooleanOptionalAction,default=False,
    help="Don't use the Natural Scenes Dataset for pretraining",
)
parser.add_argument(
    "--snr_threshold",type=float,default=-1.0,
    help="Used for calculating SNR on a whole brain to narrow down voxels.",
)
parser.add_argument(
    "--mode",type=str,default="all",
)
parser.add_argument(
    "--dual_guidance",action=argparse.BooleanOptionalAction,default=False,
    help="Use the decoded captions for dual guidance",
)

if utils.is_interactive():
    args = parser.parse_args(jupyter_args)
else:
    args = parser.parse_args()

# create global variables without the args prefix
for attribute_name in vars(args).keys():
    globals()[attribute_name] = getattr(args, attribute_name)
    
# seed all random functions
utils.seed_everything(seed)

outdir = os.path.abspath(f'../train_logs/{model_name}')
if not os.path.exists(outdir) and ckpt_saving:
    os.makedirs(outdir,exist_ok=True)
    
if use_image_aug or blurry_recon:
    import kornia
    from kornia.augmentation.container import AugmentationSequential
if use_image_aug:
    img_augment = AugmentationSequential(
        kornia.augmentation.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1, p=0.3),
        same_on_batch=False,
        data_keys=["input"],
    )
    
if multi_subject:
    if train_imageryrf:
            # 9,10,11 is ImageryRF subjects
        if no_nsd:
            subj_list = np.arange(9,12)
        else:
            subj_list = np.arange(1,12)
    else:
        subj_list = np.arange(1,9)
    subj_list = subj_list[subj_list != subj]
else:
    subj_list = [subj]

print("subj_list", subj_list, "num_sessions", num_sessions)

subj_list [1] num_sessions 40


# Prep data, models, and dataloaders

### Creating wds dataloader, preload betas and all 73k possible images

In [4]:
def my_split_by_node(urls): return urls
num_voxels_list = []
num_devices = 1
if multi_subject:
    nsessions_allsubj=np.array([40, 40, 32, 30, 40, 32, 40, 30])
    num_samples_per_epoch = (750*40) // num_devices 
else:
    num_samples_per_epoch = (750*num_sessions) // num_devices 

print("dividing batch size by subj_list, which will then be concatenated across subj during training...") 
batch_size = batch_size // len(subj_list)

num_iterations_per_epoch = num_samples_per_epoch // (batch_size*len(subj_list))

print("batch_size =", batch_size, "num_iterations_per_epoch =",num_iterations_per_epoch, "num_samples_per_epoch =",num_samples_per_epoch)



dividing batch size by subj_list, which will then be concatenated across subj during training...
batch_size = 64 num_iterations_per_epoch = 468 num_samples_per_epoch = 30000


In [5]:
train_data = {}
train_dl = {}
num_voxels = {}
voxels = {}
for s in subj_list:
    print(f"Training with {num_sessions} sessions")
    # If an NSD subject
    if s < 9:
        if multi_subject:
            train_url = f"{data_path}/wds/subj{s:02d}/train/" + "{0.." + f"{nsessions_allsubj[s-1]-1}" + "}.tar"
        else:
            train_url = f"{data_path}/wds/subj{s:02d}/train/" + "{0.." + f"{num_sessions-1}" + "}.tar"
        print(train_url)
        
        train_data[f'subj{s:02d}'] = wds.WebDataset(train_url,resampled=True,nodesplitter=my_split_by_node)\
                            .shuffle(750, initial=1500, rng=random.Random(42))\
                            .decode("torch")\
                            .rename(behav="behav.npy", past_behav="past_behav.npy", future_behav="future_behav.npy", olds_behav="olds_behav.npy")\
                            .to_tuple(*["behav", "past_behav", "future_behav", "olds_behav"])
        train_dl[f'subj{s:02d}'] = torch.utils.data.DataLoader(train_data[f'subj{s:02d}'], batch_size=batch_size, shuffle=False, drop_last=True, pin_memory=True)

        betas = utils.create_snr_betas(subject=s, data_type=data_type, data_path=data_path, threshold = snr_threshold)
        num_voxels_list.append(betas[0].shape[-1])
        num_voxels[f'subj{s:02d}'] = betas[0].shape[-1]
        voxels[f'subj{s:02d}'] = betas
    elif s < 12:
        train_url = ""
        test_url = ""
        betas, images, _, _ = utils.load_imageryrf(subject=int(s-8), mode=mode, mask=True, stimtype="object", average=False, nest=False, split=True)
        betas = torch.where(torch.isnan(betas), torch.zeros_like(betas), betas)
        betas = betas.to("cpu").to(data_type)
        num_voxels_list.append(betas[0].shape[-1])
        num_voxels[f'subj{s:02d}'] = betas[0].shape[-1]
        num_nan_values = torch.sum(torch.isnan(betas))
        print("Number of NaN values in betas:", num_nan_values.item())
        indices = torch.randperm(len(betas))
        shuffled_betas = betas[indices]
        shuffled_images = images[indices]
        train_data[f'subj{s:02d}'] = torch.utils.data.TensorDataset(shuffled_betas, shuffled_images)
        train_dl[f'subj{s:02d}'] = torch.utils.data.DataLoader(train_data[f'subj{s:02d}'], batch_size=batch_size, shuffle=False, drop_last=True, pin_memory=True)
        
        
    # elif s < 15:
    #     betas, images = utils.load_imageryrf(subject=int(s-11), mode="imagery", mask=True, stimtype="object", average=False, nest=False)
    #     betas = torch.where(torch.isnan(betas), torch.zeros_like(betas), betas)
    #     betas = betas.to("cpu").to(data_type)
    #     num_voxels_list.append(betas[0].shape[-1])
    #     num_voxels[f'subj{s:02d}'] = betas[0].shape[-1]
        
    #     indices = torch.randperm(len(betas))
    #     shuffled_betas = betas[indices]
    #     shuffled_images = images[indices]
    #     train_data[f'subj{s:02d}'] = torch.utils.data.TensorDataset(shuffled_betas, shuffled_images)
    #     train_dl[f'subj{s:02d}'] = torch.utils.data.DataLoader(train_data[f'subj{s:02d}'], batch_size=batch_size, shuffle=False, drop_last=True, pin_memory=True)
    print(f"num_voxels for subj{s:02d}: {num_voxels[f'subj{s:02d}']}")

print("Loaded all subj train dls and betas!\n")

# Validate only on one subject (doesn't support ImageryRF)
if multi_subject: 
    subj = subj_list[0] # cant validate on the actual held out person so picking first in subj_list
if not new_test: # using old test set from before full dataset released (used in original MindEye paper)
    if subj==3:
        num_test=2113
    elif subj==4:
        num_test=1985
    elif subj==6:
        num_test=2113
    elif subj==8:
        num_test=1985
    else:
        num_test=2770
    test_url = f"{data_path}/wds/subj0{subj}/test/" + "0.tar"
elif new_test: # using larger test set from after full dataset released
    if subj==3:
        num_test=2371
    elif subj==4:
        num_test=2188
    elif subj==6:
        num_test=2371
    elif subj==8:
        num_test=2188
    else:
        num_test=3000
    test_url = f"{data_path}/wds/subj0{subj}/new_test/" + "0.tar"
print(test_url)
if subj < 9:
    test_data = wds.WebDataset(test_url,resampled=False,nodesplitter=my_split_by_node)\
                        .shuffle(750, initial=1500, rng=random.Random(42))\
                        .decode("torch")\
                        .rename(behav="behav.npy", past_behav="past_behav.npy", future_behav="future_behav.npy", olds_behav="olds_behav.npy")\
                        .to_tuple(*["behav", "past_behav", "future_behav", "olds_behav"])
    test_dl = torch.utils.data.DataLoader(test_data, batch_size=num_test, shuffle=False, drop_last=True, pin_memory=True)
else:
    _, _, betas, images = utils.load_imageryrf(subject=int(subj-8), mode=mode, mask=True, stimtype="object", average=False, nest=True, split=True)
    num_test = len(betas)
    betas = torch.where(torch.isnan(betas), torch.zeros_like(betas), betas)
    betas = betas.to("cpu").to(data_type)
    num_nan_values = torch.sum(torch.isnan(betas))
    print("Number of NaN values in test betas:", num_nan_values.item())
    test_data = torch.utils.data.TensorDataset(betas, images)
    test_dl = torch.utils.data.DataLoader(test_data, batch_size=num_test, shuffle=False, drop_last=True, pin_memory=True)
print(f"Loaded test dl for subj{subj}!\n")

seq_len = seq_past + 1 + seq_future
print(f"currently using {seq_len} seq_len (chose {seq_past} past behav and {seq_future} future behav)")

Training with 40 sessions
../dataset//wds/subj01/train/{0..39}.tar
torch.Size([27000, 15724]) (27000,)
num_voxels for subj01: 15724
Loaded all subj train dls and betas!

../dataset//wds/subj01/new_test/0.tar
Loaded test dl for subj1!

currently using 1 seq_len (chose 0 past behav and 0 future behav)


In [6]:
# Load 73k NSD images
f = h5py.File(f'{data_path}/coco_images_224_float16.hdf5', 'r')
images = f['images'] # if you go OOM you can remove the [:] so it isnt preloaded to cpu! (will require a few edits elsewhere tho)
# images = torch.Tensor(images).to("cpu").to(data_type)
print("Loaded all 73k possible NSD images to cpu!", images.shape)

# Load 73k NSD captions
captions = np.load(f'{data_path}/preprocessed_data/annots_73k.npy')
print("Loaded all 73k NSD captions to cpu!", captions.shape)


Loaded all 73k possible NSD images to cpu! (73000, 3, 224, 224)
27000
Loaded train images for subj1! torch.Size([27000, 3, 224, 224])
Loaded all 73k NSD captions to cpu! (73000,)


## Load models

In [ ]:

if blurry_recon:
    from diffusers import AutoencoderKL    
    autoenc = AutoencoderKL(
        down_block_types=['DownEncoderBlock2D', 'DownEncoderBlock2D', 'DownEncoderBlock2D', 'DownEncoderBlock2D'],
        up_block_types=['UpDecoderBlock2D', 'UpDecoderBlock2D', 'UpDecoderBlock2D', 'UpDecoderBlock2D'],
        block_out_channels=[128, 256, 512, 512],
        layers_per_block=2,
        sample_size=256,
    )
    ckpt = torch.load(f'{cache_dir}/sd_image_var_autoenc.pth')
    # Create a mapping from the old layer names to the new layer names
    layer_mapping = {
        "encoder.mid_block.attentions.0.to_q.weight": "encoder.mid_block.attentions.0.query.weight",
        "encoder.mid_block.attentions.0.to_q.bias": "encoder.mid_block.attentions.0.query.bias",
        "encoder.mid_block.attentions.0.to_k.weight": "encoder.mid_block.attentions.0.key.weight",
        "encoder.mid_block.attentions.0.to_k.bias": "encoder.mid_block.attentions.0.key.bias",
        "encoder.mid_block.attentions.0.to_v.weight": "encoder.mid_block.attentions.0.value.weight",
        "encoder.mid_block.attentions.0.to_v.bias": "encoder.mid_block.attentions.0.value.bias",
        "encoder.mid_block.attentions.0.to_out.0.weight": "encoder.mid_block.attentions.0.proj_attn.weight",
        "encoder.mid_block.attentions.0.to_out.0.bias": "encoder.mid_block.attentions.0.proj_attn.bias",
        "decoder.mid_block.attentions.0.to_q.weight": "decoder.mid_block.attentions.0.query.weight",
        "decoder.mid_block.attentions.0.to_q.bias": "decoder.mid_block.attentions.0.query.bias",
        "decoder.mid_block.attentions.0.to_k.weight": "decoder.mid_block.attentions.0.key.weight",
        "decoder.mid_block.attentions.0.to_k.bias": "decoder.mid_block.attentions.0.key.bias",
        "decoder.mid_block.attentions.0.to_v.weight": "decoder.mid_block.attentions.0.value.weight",
        "decoder.mid_block.attentions.0.to_v.bias": "decoder.mid_block.attentions.0.value.bias",
        "decoder.mid_block.attentions.0.to_out.0.weight": "decoder.mid_block.attentions.0.proj_attn.weight",
        "decoder.mid_block.attentions.0.to_out.0.bias": "decoder.mid_block.attentions.0.proj_attn.bias"
    }

    # Create a new state dictionary with the renamed layers
    new_ckpt = {}
    for old_key, value in ckpt.items():
        new_key = layer_mapping.get(old_key, old_key)  # Get the new key, or use the old key if not in mapping
        new_ckpt[new_key] = value
    autoenc.load_state_dict(new_ckpt)
    
    autoenc.eval()
    autoenc.requires_grad_(False)
    autoenc.to(device)
    utils.count_params(autoenc)

### CLIP image embeddings  model

In [7]:
clip_emb_dim = 768
clip_seq_dim = 257
clip_text_seq_dim=77
clip_extractor = Reconstructor(device=device, cache_dir=cache_dir)
clip_variant = "ViT-L-14"
output_dims = {}

Creating Clipper...


/export/raid1/home/kneel027/mamba/envs/mindeye_imagery_turbo/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Creating block of CLIP embeddings

In [10]:
file_path = f"{data_path}/preprocessed_data/{clip_variant}_image_embeddings.pt"

if not os.path.exists(file_path):
    # Generate CLIP Image embeddings
    print("Generating CLIP Image embeddings!")
    clip_image = torch.zeros((len(images), clip_seq_dim * clip_emb_dim)).to("cpu")
    for i in tqdm(range(0, len(images)), desc="Encoding images..."):
        clip_image[i] = clip_extractor.embed_image(torch.from_numpy(images[i]).unsqueeze(0)).flatten().detach().to("cpu")
    torch.save(clip_image, file_path)
else:
    clip_image = torch.load(file_path)
output_dims["image"] = int(clip_image.shape[-1])
    
if dual_guidance:
    file_path_txt = f"{data_path}/preprocessed_data/{clip_variant}_text_embeddings.pt"
    if not os.path.exists(file_path_txt):
        # Generate CLIP Image embeddings
        print("Generating CLIP Text embeddings!")
        clip_text = torch.zeros((len(captions), clip_text_seq_dim * clip_emb_dim)).to("cpu")
        for i in tqdm(range(0, len(captions)), desc="Encoding captions..."):
            clip_text[i] = clip_extractor.embed_text(str(captions[i])).flatten().detach().to("cpu")
        torch.save(clip_text, file_path_txt)
    else:
        clip_text = torch.load(file_path_txt)
    output_dims["text"] = int(clip_text.shape[-1])
    
if blurry_recon:
    file_path = f"{data_path}/preprocessed_data/autoenc_image_embeddings.pt"
    if not os.path.exists(file_path):
        # Generate CLIP Image embeddings
        print("Generating VAE Image embeddings!")
        vae_image = torch.zeros((len(images), 3136)).to("cpu")
        with torch.cuda.amp.autocast(dtype=torch.float16):
            for i in tqdm(range(0, len(images)), desc="Encoding images..."):
                vae_image[i] = (autoenc.encode(2*torch.from_numpy(images[i]).unsqueeze(0).detach().to(device=device, dtype=torch.float16)-1).latent_dist.mode() * 0.18215).detach().to("cpu").flatten()
            torch.save(vae_image, file_path)
    else:
        vae_image = torch.load(file_path)
    output_dims["blurry"] = int(vae_image.shape[-1])

100%|██████████| 540/540 [03:52<00:00,  2.32it/s]


# Initialize model class

In [ ]:
class MindEye_Imagery_Ridge(nn.Module):
    def __init__(self, input_dim, output_dims, hidden_dim=1024, dual_guidance=False, blurry_recon=False):
        super(MindEye_Imagery_Ridge, self).__init__()
        self.dual_guidance = dual_guidance
        self.blurry_recon = blurry_recon
        self.input_layer = nn.Linear(input_dim, hidden_dim, bias=True)
        # Image linear layer
        self.image_layer = nn.Linear(hidden_dim, output_dims['image'], bias=True)

        # Text linear layer (if enabled)
        if self.dual_guidance:
            self.text_layer = nn.Linear(hidden_dim, output_dims['text'], bias=True)

        # Blurry recon linear layer (if enabled)
        if self.blurry_recon:
            self.blurry_layer = nn.Linear(hidden_dim, output_dims['blurry'], bias=True)

    def forward(self, x):
        x = self.input_layer(x)
        outputs = {'image': self.image_layer(x)}
        if self.dual_guidance:
            outputs['text'] = self.text_layer(x)
        if self.blurry_recon:
            outputs['blurry'] = self.blurry_layer(x)
        return outputs

model = MindEye_Imagery_Ridge(num_voxels[f'subj{subj:02d}'], output_dims, hidden_dim, dual_guidance, blurry_recon)
model.to(device)

# Prepare models and utils

In [9]:
# Define optimizer with weight decay (L2 regularization)
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']

opt_grouped_parameters = [{'params': [p for n, p in model.named_parameters()], 'weight_decay': weight_decay}]
    
optimizer = torch.optim.AdamW(opt_grouped_parameters, lr=max_lr)

lr_scheduler = torch.optim.lr_scheduler.LinearLR(
    optimizer,
    total_iters=int(np.floor(num_epochs*num_iterations_per_epoch)),
    last_epoch=-1
)

def save_ckpt(tag):
    ckpt_path = outdir+f'/{tag}.pth'
    if accelerator.is_main_process:
        unwrapped_model = accelerator.unwrap_model(model)
        torch.save({
            'epoch': epoch,
            'model_state_dict': unwrapped_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'lr_scheduler': lr_scheduler.state_dict(),
            'train_losses': losses,
            'test_losses': test_losses,
            'lrs': lrs,
            }, ckpt_path)
    print(f"\n---saved {outdir}/{tag} ckpt!---\n")

def load_ckpt(tag,load_lr=True,load_optimizer=True,load_epoch=True,strict=True,outdir=outdir,multisubj_loading=False): 
    print(f"\n---loading {outdir}/{tag}.pth ckpt---\n")
    checkpoint = torch.load(outdir+'/last.pth', map_location='cpu')
    state_dict = checkpoint['model_state_dict']
    if multisubj_loading: # remove incompatible ridge layer that will otherwise error
        state_dict.pop('ridge.linears.0.weight',None)
    model.load_state_dict(state_dict, strict=strict)
    if load_epoch:
        globals()["epoch"] = checkpoint['epoch']
        print("Epoch",epoch)
    if load_optimizer:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if load_lr:
        lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
    del checkpoint

print("\nDone with model preparations!")
num_params = utils.count_params(model)

torch.Size([27000, 15724])
Number of NaN values in betas: 0


/export/raid1/home/kneel027/mamba/envs/mindeye_imagery_turbo/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
if local_rank==0 and wandb_log: # only use main process for wandb logging
    import wandb
    print(f"wandb {wandb_project} run {model_name}")
    # need to configure wandb beforehand in terminal with "wandb init"!
    wandb_config = {
      "model_name": model_name,
      "use-prior": use_prior,
      "blurry_recon": blurry_recon,
      "global_batch_size": global_batch_size,
      "batch_size": batch_size,
      "num_epochs": num_epochs,
      "num_sessions": num_sessions,
      "num_params": num_params,
      "clip_scale": clip_scale,
      "prior_scale": prior_scale,
      "blur_scale": blur_scale,
      "use_image_aug": use_image_aug,
      "max_lr": max_lr,
      "mixup_pct": mixup_pct,
      "num_samples_per_epoch": num_samples_per_epoch,
      "num_test": num_test,
      "ckpt_interval": ckpt_interval,
      "ckpt_saving": ckpt_saving,
      "seed": seed,
      "distributed": distributed,
      "num_devices": num_devices,
      "world_size": world_size,
      "train_url": train_url,
      "test_url": test_url,
      "train_imageryrf": train_imageryrf,
      "mode": mode,
    }
    print("wandb_config:\n",wandb_config)
    print("wandb_id:",model_name)
    wandb.init(
        id=model_name,
        project=wandb_project,
        name=model_name,
        config=wandb_config,
        resume=None,
    )
else:
    wandb_log = False

In [ ]:
epoch = 0
losses, test_losses, lrs = [], [], []
best_test_loss = 1e9
torch.cuda.empty_cache()

In [ ]:
train_dls = [train_dl[f'subj{s:02d}'] for s in subj_list]

model, optimizer, *train_dls = accelerator.prepare(model, optimizer, *train_dls, lr_scheduler)
# leaving out test_dl since we will only have local_rank 0 device do evals

In [ ]:
print(f"{model_name} starting with epoch {epoch} / {num_epochs}")
progress_bar = tqdm(range(epoch,num_epochs), ncols=1200, disable=(local_rank!=0))
test_image, test_voxel = None, None
test_caption = []
mse = nn.MSELoss()
l1 = nn.L1Loss()
soft_loss_temps = utils.cosine_anneal(0.004, 0.0075, num_epochs - int(mixup_pct * num_epochs))

for epoch in progress_bar:
    model.train()

    fwd_percent_correct = 0.
    bwd_percent_correct = 0.
    test_fwd_percent_correct = 0.
    test_bwd_percent_correct = 0.
    
    recon_cossim = 0.
    recon_cossim_txt = 0.
    test_recon_cossim = 0.
    test_recon_cossim_txt = 0.
    recon_mse = 0.
    test_recon_mse = 0.

    loss_image_clip_total = 0.
    loss_text_clip_total = 0.
    loss_blurry_total = 0.
    
    test_loss_image_clip_total = 0.
    test_loss_text_clip_total = 0.
    test_loss_blurry_total = 0.
    
    loss_prior_total = 0.
    loss_prior_total_txt = 0.
    test_loss_prior_total = 0.
    test_loss_prior_total_txt = 0.
    loss_blurry_cont_total = 0.
    test_loss_blurry_cont_total = 0.
    blurry_pixcorr = 0.
    test_blurry_pixcorr = 0. # needs >.456 to beat low-level subj01 results in mindeye v1

    # pre-load all batches for this epoch (it's MUCH faster to pre-load in bulk than to separate loading per batch)
    voxel_iters = {} # empty dict because diff subjects have differing # of voxels
    image_iters = torch.zeros(num_iterations_per_epoch, batch_size*len(subj_list), 3, 224, 224).float()
    image_clip_iters = torch.zeros(num_iterations_per_epoch, batch_size*len(subj_list), clip_seq_dim * clip_emb_dim).float()
    if dual_guidance:
        text_clip_iters = torch.zeros(num_iterations_per_epoch, batch_size*len(subj_list), clip_text_seq_dim * clip_emb_dim).float()
    if blurry_recon:
        vae_image_iters = torch.zeros(num_iterations_per_epoch, batch_size*len(subj_list), 3136).float()

    perm_iters, betas_iters, select_iters = {}, {}, {}
    # print(f"num_iterations_per_epoch: {num_iterations_per_epoch}, batch_size: {batch_size}, len(subj_list): {len(subj_list)}")
    for s, (cur_subj, train_dl) in enumerate(zip(subj_list, train_dls)):
        with torch.cuda.amp.autocast(dtype=data_type):
            i = 0
            while i < num_iterations_per_epoch:
                # print(f"restarting data loader at i={i} for s={s}")
                for data in train_dl:  
                    if cur_subj < 9:
                        behav0, past_behav0, future_behav0, old_behav0 = data
                        
                        # image0 = images[behav0[:,0,0].cpu().long()].float()
                        # image_sorted_idx = behav0[:,0,0].cpu().long().numpy()
                        # image_sorted_idx = np.unique(np.sort(image_sorted_idx))
                        
                        # image0 = images[image_sorted_idx]
                        # image0 = torch.tensor(image0, dtype=torch.float16, device="cpu")  # Convert to tensor
                        # while image0.shape[0] < batch_size:
                        #     image0 = torch.cat((image0, image0[0].unsqueeze(0)), dim=0)
                        image_idx = behav0[:,0,0].cpu().long().numpy()
                        local_idx, image_sorted_idx = np.unique(image_idx, return_index=True)                
                        # if len(image0) != len(image_idx): # hdf5 cant handle duplicate indexing
                        #     continue
                        image0 = torch.tensor(images[local_idx], dtype=data_type)
                        image_iters[i, s*batch_size:s*batch_size+image0.shape[0]] = image0
                        image_clip0 = clip_image[local_idx]
                        image_clip_iters[i, s*batch_size:s*batch_size+image_clip0.shape[0]]
                        
                        if dual_guidance:
                            text_clip0 = clip_text[local_idx]
                            text_clip_iters[i, s*batch_size:s*batch_size+text_clip0.shape[0]]
                        if blurry_recon:
                            vae_image0 = vae_image[local_idx]
                            vae_image_iters[i, s*batch_size:s*batch_size+vae_image0.shape[0]]
                        
                        voxel_idx = behav0[:,0,5].cpu().long().numpy()
                        voxel_sorted_idx = voxel_idx[image_sorted_idx]
                        voxel0 = voxels[f'subj0{subj_list[s]}'][voxel_sorted_idx]
                        voxel0 = torch.Tensor(voxel0)#.unsqueeze(1)
                    else:
                        voxel0, image0 = data
                        image0 = torch.nn.functional.interpolate(image0, size=(224, 224), mode='bilinear', align_corners=False)
                        image_iters[i,s*batch_size:s*batch_size+batch_size] = image0
                    
                    if seq_len==1:
                        voxel0 = voxel0.unsqueeze(1)
                    else:
                        if seq_past>0:
                            past_behavior = past_behav0[:,:(seq_past),5].cpu().long()
                            past_voxel0 = voxels[f'subj{subj_list[s]:02d}'][past_behavior]
                            past_voxel0[past_behavior==-1] = voxel0[torch.where(past_behavior==-1)[0]] # replace invalid past voxels 
                            past_voxel0 = torch.Tensor(past_voxel0)

                            # if shared1000, then you need to mask it out 
                            for p in range(seq_past):
                                mask = (past_behav0[:,p,-1] == 1) # [16,] bool
                                index = torch.nonzero(mask.cpu()).squeeze()
                                past_voxel0[index,p,:] = torch.zeros_like(past_voxel0[index,p,:])

                        if seq_future>0:
                            future_behavior = future_behav0[:,:(seq_future),5].cpu().long()
                            future_voxel0 = voxels[f'subj{subj_list[s]:02d}'][future_behavior]
                            future_voxel0[future_behavior==-1] = voxel0[torch.where(future_behavior==-1)[0]] # replace invalid past voxels 
                            future_voxel0 = torch.Tensor(future_voxel0)

                            # if shared1000, then you need to mask it out 
                            for p in range(seq_future):
                                mask = (future_behav0[:,p,-1] == 1) # [16,] bool
                                index = torch.nonzero(mask.cpu()).squeeze()
                                future_voxel0[index,p,:] = torch.zeros_like(future_voxel0[index,p,:])

                        # concatenate current timepoint with past/future
                        if seq_past > 0 and seq_future > 0:
                            voxel0 = torch.cat((voxel0.unsqueeze(1), past_voxel0), axis=1)
                            voxel0 = torch.cat((voxel0, future_voxel0), axis=1)
                        elif seq_past > 0:
                            voxel0 = torch.cat((voxel0.unsqueeze(1), past_voxel0), axis=1)
                        else:
                            voxel0 = torch.cat((voxel0.unsqueeze(1), future_voxel0), axis=1)

                    if epoch < int(mixup_pct * num_epochs):
                        voxel0, perm, betas, select = utils.mixco(voxel0)
                        perm_iters[f"subj{subj_list[s]:02d}_iter{i}"] = perm
                        betas_iters[f"subj{subj_list[s]:02d}_iter{i}"] = betas
                        select_iters[f"subj{subj_list[s]:02d}_iter{i}"] = select

                    voxel_iters[f"subj{subj_list[s]:02d}_iter{i}"] = voxel0
                    i +=1
                    if ~(i<num_iterations_per_epoch):
                        # print(f"breaking data loader at i={i} for s={s}")
                        break
                    
                    
                    

    # you now have voxel_iters and image_iters with num_iterations_per_epoch batches each
    for train_i in range(num_iterations_per_epoch):
        with torch.cuda.amp.autocast(dtype=data_type):
            optimizer.zero_grad()
            loss=0.

            voxel_list = [voxel_iters[f"subj{s:02d}_iter{train_i}"] for s in subj_list]
            train_voxels = voxel_list[0].detach().to(device)
            # print(f"voxel_list {voxel_list}")
            image = image_iters[train_i][:train_voxels.shape[0]].detach()
            image_clip_target = image_clip_iters[train_i][:train_voxels.shape[0]].detach().to(device)
            image_clip_target_norm = nn.functional.normalize(image_clip_target.flatten(1), dim=-1)
            if dual_guidance:
                text_clip_target = text_clip_iters[train_i][:train_voxels.shape[0]].detach().to(device)
                text_clip_target_norm = nn.functional.normalize(text_clip_target.flatten(1), dim=-1)
            if blurry_recon:
                vae_image_target = vae_image_iters[train_i][:train_voxels.shape[0]].detach().to(device)
                vae_image_target_norm = nn.functional.normalize(vae_image_target.flatten(1), dim=-1)

            outputs = model(train_voxels)
            image_clip_voxels_norm = nn.functional.normalize(outputs["image"].flatten(1), dim=-1)
            image_clip_loss = mse(image_clip_voxels_norm, image_clip_target_norm.unsqueeze(1))
            loss += image_clip_loss
            loss_image_clip_total += image_clip_loss.item()
            if dual_guidance:
                text_clip_voxels_norm = nn.functional.normalize(outputs["text"].flatten(1), dim=-1)
                text_clip_loss = mse(text_clip_voxels_norm, text_clip_target_norm.unsqueeze(1))
                loss += text_clip_loss
                loss_text_clip_total += text_clip_loss.item()
            if blurry_recon:
                blurry_image_enc_norm = nn.functional.normalize(outputs["blurry"].flatten(1), dim=-1)
                blurry_image_loss = mse(blurry_image_enc_norm, vae_image_target_norm.unsqueeze(1))
                loss += blurry_image_loss
                loss_blurry_total += blurry_image_loss.item()
                
            if clip_scale>0:
                # forward and backward top 1 accuracy        
                labels = torch.arange(len(image_clip_voxels_norm)).to(image_clip_voxels_norm.device) 
                fwd_percent_correct += utils.topk(utils.batchwise_cosine_similarity(image_clip_voxels_norm, image_clip_target_norm), labels, k=1).item()
                bwd_percent_correct += utils.topk(utils.batchwise_cosine_similarity(image_clip_target_norm, image_clip_voxels_norm), labels, k=1).item()

            if blurry_recon:
                with torch.no_grad():
                    # only doing pixcorr eval on a subset of the samples per batch because its costly & slow to compute autoenc.decode()
                    random_samps = np.random.choice(np.arange(len(image)), size=len(image)//5, replace=False)
                    try:
                        blurry_recon_images = (autoenc.decode(blurry_image_enc[random_samps].reshape((-1, 4, 28, 28))/0.18215).sample/ 2 + 0.5).clamp(0,1)
                        pixcorr = utils.pixcorr(image[random_samps].cpu().float(), blurry_recon_images.cpu().float())
                    except:
                        print(random_samps, blurry_image_enc.shape)
                        print(blurry_image_enc[random_samps].reshape((-1, 4, 28, 28)).shape, image[random_samps].shape)
                        print(blurry_recon_images.shape)
                        sys.exit()
                    blurry_pixcorr += pixcorr.item()

            utils.check_loss(loss)
            accelerator.backward(loss)
            optimizer.step()

            losses.append(loss.item())
            lrs.append(optimizer.param_groups[0]['lr'])

            if lr_scheduler_type is not None:
                lr_scheduler.step()

    model.eval()
    if local_rank==0:
        with torch.no_grad(), torch.cuda.amp.autocast(dtype=data_type): 
            caption = []
            for test_i, data in enumerate(test_dl):  
                if subj < 9:
                    behav, past_behav, future_behav, old_behav = data
                    # all test samples should be loaded per batch such that test_i should never exceed 0
                    assert len(behav) == num_test

                    ## Average same-image repeats ##
                    if test_image is None:
                        voxel = voxels[f'subj0{subj}'][behav[:,0,5].cpu().long().numpy()]
                        
                        if seq_len==1:
                            voxel = voxel.unsqueeze(1)
                        else:
                            if seq_past>0:
                                past_behavior = past_behav[:,:(seq_past),5].cpu().long()
                                past_voxels = voxels[f'subj0{subj}'][past_behavior]
                                if torch.any(past_behavior==-1).item(): # remove invalid voxels (-1 if there is no timepoint available)
                                    past_voxels[torch.where(past_behavior==-1)[0]] = 0

                            if seq_future>0:
                                future_behavior = future_behav[:,:(seq_future),5].cpu().long()
                                future_voxels = voxels[f'subj0{subj}'][future_behavior]                    
                                if torch.any(future_behavior==-1).item(): # remove invalid voxels (-1 if there is no timepoint available)
                                    future_voxels[torch.where(future_behavior==-1)[0]] = 0
                                
                            if seq_past > 0 and seq_future > 0:
                                voxel = torch.cat((voxel.unsqueeze(1), past_voxels), axis=1)
                                voxel = torch.cat((voxel, future_voxels), axis=1)
                            elif seq_past > 0:
                                voxel = torch.cat((voxel.unsqueeze(1), past_voxels), axis=1)
                            else:
                                voxel = torch.cat((voxel.unsqueeze(1), future_voxels), axis=1)

                        image = behav[:,0,0].cpu().long()

                        unique_image, sort_indices = torch.unique(image, return_inverse=True)
                        for im in unique_image:
                            locs = torch.where(im == image)[0]
                            if len(locs)==1:
                                locs = locs.repeat(3)
                            elif len(locs)==2:
                                locs = locs.repeat(2)[:3]
                            assert len(locs)==3
                            if test_image is None:
                                test_image = torch.tensor(images[im][None], dtype=torch.float16, device="cpu")
                                test_image_clip = clip_image[im][None]
                                if dual_guidance:
                                    test_text_clip = clip_text[im][None]
                                if blurry_recon:
                                    test_vae_image = vae_image[im][None]
                                test_voxel = voxel[locs][None]
                            else:
                                test_image = torch.vstack((test_image, torch.tensor(images[im][None], dtype=torch.float16, device="cpu")))
                                test_image_clip = torch.vstack((test_image_clip, clip_image[im][None]))
                                if dual_guidance:
                                    test_text_clip = torch.vstack((test_text_clip, clip_text[im][None]))
                                if blurry_recon:
                                    test_vae_image = torch.vstack((test_vae_image, vae_image[im][None]))
                                test_voxel = torch.vstack((test_voxel, voxel[locs][None]))
                                
                    test_indices = torch.arange(len(test_voxel))[:300]
                    # print(test_image.shape,len(test_caption),test_indices.max())
                    voxel = test_voxel[test_indices].to(device)
                    image = test_image[test_indices].to(device)
                    image_clip_target = test_image_clip[test_indices].to(device)
                    image_clip_target_norm = nn.functional.normalize(image_clip_target.flatten(1), dim=-1)
                    if dual_guidance:
                        text_clip_target = test_text_clip[test_indices].to(device)
                        text_clip_target_norm = nn.functional.normalize(text_clip_target.flatten(1), dim=-1)
                    if blurry_recon:
                        vae_image_target = test_vae_image[test_indices].to(device)
                        vae_image_target_norm = nn.functional.normalize(vae_image_target.flatten(1), dim=-1)
                else:
                    voxel, image = data
                    voxel = voxel.unsqueeze(2).to(device)
                    image = torch.nn.functional.interpolate(image, size=(224, 224), mode='bilinear', align_corners=False)
                    image = image.to(device)
                
                loss=0.
                voxel = torch.mean(voxel, dim=1).to(device)
                
                outputs = model(voxel)
                image_clip_voxels = outputs["image"]
                image_clip_voxels_norm = nn.functional.normalize(image_clip_voxels.flatten(1), dim=-1)
                image_clip_loss = mse(image_clip_voxels_norm, image_clip_target_norm.unsqueeze(1))
                loss += image_clip_loss
                test_loss_image_clip_total += image_clip_loss.item()
                if dual_guidance:
                    text_clip_voxels = outputs["text"]
                    text_clip_voxels_norm = nn.functional.normalize(text_clip_voxels.flatten(1), dim=-1)
                    text_clip_loss = mse(text_clip_voxels_norm, text_clip_target_norm.unsqueeze(1))
                    loss += text_clip_loss
                    test_loss_text_clip_total += text_clip_loss.item()
                if blurry_recon:
                    blurry_image_enc = outputs["blurry"]
                    blurry_image_enc_norm = nn.functional.normalize(blurry_image_enc.flatten(1), dim=-1)
                    blurry_image_loss = mse(blurry_image_enc_norm, vae_image_target_norm.unsqueeze(1))
                    loss += blurry_image_loss
                    test_loss_blurry_total += blurry_image_loss.item()
                    
                if clip_scale>0:
                    image_clip_voxels_norm = nn.functional.normalize(image_clip_voxels.flatten(1), dim=-1)
                    image_clip_target_norm = nn.functional.normalize(image_clip_target.flatten(1), dim=-1)

                    # forward and backward top 1 accuracy        
                    labels = torch.arange(len(image_clip_voxels_norm)).to(image_clip_voxels_norm.device) 
                    test_fwd_percent_correct += utils.topk(utils.batchwise_cosine_similarity(image_clip_voxels_norm, image_clip_target_norm), labels, k=1).item()
                    test_bwd_percent_correct += utils.topk(utils.batchwise_cosine_similarity(image_clip_target_norm, image_clip_voxels_norm), labels, k=1).item()

                if blurry_recon:
                    with torch.no_grad():
                        # only doing pixcorr eval on a subset of the samples per batch because its costly & slow to compute autoenc.decode()
                        random_samps = np.random.choice(np.arange(len(image)), size=len(image)//5, replace=False)
                        blurry_recon_images = (autoenc.decode(blurry_image_enc.reshape((-1, 4, 28, 28))[random_samps]/0.18215).sample/ 2 + 0.5).clamp(0,1)
                        pixcorr = utils.pixcorr(image[random_samps].cpu().float(), blurry_recon_images.cpu().float())
                        test_blurry_pixcorr += pixcorr.item()
                
                utils.check_loss(loss)                
                test_losses.append(loss.item())

            # if utils.is_interactive(): clear_output(wait=True)
            print("---")

            assert (test_i+1) == 1
            logs = {"train/loss": np.mean(losses[-(train_i+1):]),
                "test/loss": np.mean(test_losses[-(test_i+1):]),
                "train/lr": lrs[-1],
                "train/num_steps": len(losses),
                "test/num_steps": len(test_losses),
                "train/fwd_pct_correct": fwd_percent_correct / (train_i + 1),
                "train/bwd_pct_correct": bwd_percent_correct / (train_i + 1),
                "test/test_fwd_pct_correct": test_fwd_percent_correct / (test_i + 1),
                "test/test_bwd_pct_correct": test_bwd_percent_correct / (test_i + 1),
                "train/loss_image_clip_total": loss_image_clip_total / (train_i + 1),
                "train/loss_text_clip_total": loss_text_clip_total / (train_i + 1),
                "train/loss_blurry_total": loss_blurry_total / (train_i + 1),
                "train/loss_blurry_cont_total": loss_blurry_cont_total / (train_i + 1),
                "test/loss_image_clip_total": test_loss_image_clip_total / (test_i + 1),
                "test/loss_text_clip_total": test_loss_text_clip_total / (test_i + 1),
                "train/blurry_pixcorr": blurry_pixcorr / (train_i + 1),
                "test/blurry_pixcorr": test_blurry_pixcorr / (test_i + 1),
                "train/recon_cossim": recon_cossim / (train_i + 1),
                "train/recon_txt_cossim": recon_cossim_txt / (train_i +1),
                "test/recon_cossim": test_recon_cossim / (test_i + 1),
                "test/recon_txt_cossim": test_recon_cossim_txt / (test_i +1),
                "train/recon_mse": recon_mse / (train_i + 1),
                "test/recon_mse": test_recon_mse / (test_i + 1),
                "train/loss_prior": loss_prior_total / (train_i + 1),
                "test/loss_prior": test_loss_prior_total / (test_i + 1),
                "train/loss_prior_txt": loss_prior_total_txt / (train_i + 1),
                "test/loss_prior_txt": test_loss_prior_total_txt / (test_i + 1),
                }

            # if finished training, save jpg recons if they exist
            if (epoch == num_epochs-1) or (epoch % ckpt_interval == 0):
                if blurry_recon:    
                    image_enc = autoenc.encode(2*image[:4]-1).latent_dist.mode() * 0.18215
                    # transform blurry recon latents to images and plot it
                    fig, axes = plt.subplots(1, 8, figsize=(10, 4))
                    jj=-1
                    for j in [0,1,2,3]:
                        jj+=1
                        axes[jj].imshow(utils.torch_to_Image((autoenc.decode(image_enc[[j]]/0.18215).sample / 2 + 0.5).clamp(0,1)))
                        axes[jj].axis('off')
                        jj+=1
                        axes[jj].imshow(utils.torch_to_Image((autoenc.decode(blurry_image_enc[[j]].reshape((-1, 4, 28, 28))/0.18215).sample / 2 + 0.5).clamp(0,1)))
                        axes[jj].axis('off')

                    if wandb_log:
                        logs[f"test/blur_recons"] = wandb.Image(fig, caption=f"epoch{epoch:03d}")
                        plt.close()
                    else:
                        plt.show()
                        
                if use_prior and visualize_prior: # output recons every ckpt
                    idx = np.random.randint(0, 3)
                    print(f"reconstructing... idx={idx}")
                    samples = utils.unclip_recon(prior_out[[idx]],
                             diffusion_engine,
                             vector_suffix)
                    if wandb_log:
                        logs[f"test/orig"] = wandb.Image(transforms.ToPILImage()(image[idx]),
                                                           caption=f"epoch{epoch:03d}")
                        logs[f"test/recons"] = wandb.Image(transforms.ToPILImage()(samples[0]),
                                                           caption=f"epoch{epoch:03d}")
                    if utils.is_interactive():
                        plt.figure(figsize=(2,2))
                        plt.imshow(transforms.ToPILImage()(image[idx]))
                        plt.axis('off')
                        plt.show()
                        
                        plt.figure(figsize=(2,2))
                        plt.imshow(transforms.ToPILImage()(samples[0]))
                        plt.axis('off')
                        plt.show()

            progress_bar.set_postfix(**logs)

            if wandb_log: wandb.log(logs)
            
    # Save model checkpoint and reconstruct
    if (ckpt_saving) and (epoch % ckpt_interval == 0):
        save_ckpt(f'last')

    # wait for other GPUs to catch up if needed
    accelerator.wait_for_everyone()
    torch.cuda.empty_cache()
    gc.collect()

print("\n===Finished!===\n")
if ckpt_saving:
    save_ckpt(f'last')
